In [ ]:
# Instalação das dependências necessárias
!pip install "tensorflow[and-cuda]" tensorflow-hub
!pip install -q tensorflow tensorflow_hub
!pip install -q scikit-learn pandas numpy tqdm

# Para usar o BGE-M3
!pip install sentence-transformers

!pip install spacy
!python -m spacy download pt_core_news_lg

In [ ]:
# Verificação da instalação do TensorFlow com suporte a GPU
import tensorflow as tf, pprint
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))
pprint.pprint(tf.sysconfig.get_build_info())  # mostra cuda_version e cudnn_version

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import spacy
from sklearn.preprocessing import StandardScaler

#Para usar o BGE-M3
from sentence_transformers import SentenceTransformer 

In [ ]:
# Carregando os dados
df = pd.read_csv('eventos_artigos_filtrados.csv');
df.shape
#df.head()

In [3]:
# Preparação dos dados
df = df.drop(columns=['evento_nome', 'resumo'])
df = df.dropna(subset=['titulo_artigo', 'ano_edicao'])
df['antes_2023'] = df['ano_edicao'].apply(lambda x: 1 if x < 2023 else 0)

df["titulo_artigo"] = df["titulo_artigo"].astype(str).str.strip()
df["antes_2023"] = df["antes_2023"].astype(int).values

#df.head()

In [ ]:
#Incluir aqui outros tratamentos, SFC, normalização, etc.

# Geração do embeddings com BGE-M3 e posterior complementação com features de estilo

In [ ]:
titulos = df['titulo_artigo'].tolist()
labels = df['antes_2023'].values

# Carregamento do Modelo BGE-M3 ---
model = SentenceTransformer("BAAI/bge-m3")
print("BGE-M3 carregado com sucesso.")

# Execução da Geração de Embeddings ---
embeddings_gerados = model.encode(
    titulos,
    batch_size=32,
    show_progress_bar=True
)

print(f"\nEmbeddings gerados com sucesso! Shape final: {embeddings_gerados.shape}")

BGE-M3 carregado com sucesso.


Batches: 100%|██████████| 790/790 [00:41<00:00, 18.91it/s]



Embeddings gerados com sucesso! Shape final: (12629, 1024)


# Features de estilo

In [5]:
nlp = spacy.load("pt_core_news_lg")

# Gerar features
def calcular_features_estilo(texto: str):
    doc = nlp(texto)
    
    # Features de Comprimento
    num_chars = len(texto)
    num_words = len([token for token in doc if not token.is_punct])
    avg_word_len = np.mean([len(token.text) for token in doc if not token.is_punct]) if num_words > 0 else 0
    
    # Features Lexicais
    # Contagem de tipos de palavras (Part-of-Speech tagging)
    pos_counts = doc.count_by(spacy.attrs.POS)
    num_nouns = pos_counts.get(nlp.vocab.strings["NOUN"], 0) # Substantivos
    num_verbs = pos_counts.get(nlp.vocab.strings["VERB"], 0) # Verbos
    num_adjs = pos_counts.get(nlp.vocab.strings["ADJ"], 0)  # Adjetivos
    num_advs = pos_counts.get(nlp.vocab.strings["ADV"], 0)  # Advérbios
    
    # Outras Features
    num_stopwords = len([token for token in doc if token.is_stop])
    stopword_ratio = num_stopwords / num_words if num_words > 0 else 0
    num_punctuations = len([token for token in doc if token.is_punct])
    
    # Retorna um array NumPy com todas as features calculadas
    return np.array([
        num_chars, num_words, avg_word_len,
        num_nouns, num_verbs, num_adjs, num_advs,
        num_stopwords, stopword_ratio, num_punctuations
    ])

In [7]:
stylistic_features = np.array([calcular_features_estilo(titulo) for titulo in tqdm(titulos)])
scaler = StandardScaler()
stylistic_features_scaled = scaler.fit_transform(stylistic_features)

# Features Semânticas e de Estilo ---
X_combined = np.concatenate([embeddings_gerados, stylistic_features_scaled], axis=1)
y = labels

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42, stratify=y)
input_shape = X_train.shape[1] # Pega o novo número de colunas (1024 + num_features_estilo)

100%|██████████| 12629/12629 [00:46<00:00, 274.46it/s]


# CLASSIFICADOR 

In [9]:
# --- Carregamento dos Dados Pré-processados ---

print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.\n")

# Construção do Modelo de Classificação
model = tf.keras.Sequential([
    # A camada de entrada espera vetores com 1024 dimensões (o tamanho do embedding ELMo)
    tf.keras.layers.InputLayer(input_shape=(input_shape,)),
    
    # Camada oculta para aprender padrões nos embeddings
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5), # Dropout para evitar overfitting
    
    # Camada de saída para classificação binária
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# --- Compilação do Modelo ---
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Treinamento do Modelo ---
print("\nIniciando o treinamento do classificador...")
history = model.fit(
    X_train,
    y_train,
    epochs=45,
    batch_size=4,
    validation_data=(X_test, y_test),
    verbose=1
)

# --- Avaliação Final ---
print("\nAvaliação final no conjunto de teste:")
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia final no teste: {accuracy:.4f}')

Dados divididos: 10103 para treino, 2526 para teste.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 128)            │       132,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 132,609 (518.00 KB)

 Trainable params: 132,609 (518.00 KB)

...


Iniciando o treinamento do classificador...
Epoch 1/45


2025-09-04 23:47:39.777777: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f5a68007410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-04 23:47:39.777789: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-09-04 23:47:39.794987: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-04 23:47:39.889627: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


 190/2526 ━━━━━━━━━━━━━━━━━━━━ 1s 801us/step - accuracy: 0.7165 - loss: 0.5958

I0000 00:00:1757044060.508191  252935 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2526/2526 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7294 - loss: 0.5464 - val_accuracy: 0.7292 - val_loss: 0.5289
Epoch 2/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7403 - loss: 0.5183 - val_accuracy: 0.7383 - val_loss: 0.5207
Epoch 3/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7513 - loss: 0.5037 - val_accuracy: 0.7391 - val_loss: 0.5231
Epoch 4/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 994us/step - accuracy: 0.7544 - loss: 0.4926 - val_accuracy: 0.7435 - val_loss: 0.5274
Epoch 5/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7629 - loss: 0.4822 - val_accuracy: 0.7431 - val_loss: 0.5285
Epoch 6/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7724 - loss: 0.4697 - val_accuracy: 0.7427 - val_loss: 0.5293
Epoch 7/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7752 - loss: 0.4588 - val_accuracy: 0.7411 - val_loss: 0.5294
Epoch 8/45
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 995us/step - accuracy: 0.7882 - loss: 0.4473 - val_acc

In [10]:
model.save('meu_classificador_com_BGE-M3+Estilo.keras')

# TESTE COM DADOS DIFERENTES DO DATASET A PARIR DE UM MODELO GERADO.

In [11]:
# 1. Carregar o Modelo de Embedding (BGE-M3)
embedding_model = SentenceTransformer("BAAI/bge-m3")

# 2. Carregar o seu Classificador Keras treinado
classificador = tf.keras.models.load_model('meu_classificador_com_BGE-M3+Estilo.keras')

novos_titulos = [
    "Um estudo sobre o impacto da IA generativa no desenvolvimento de software",
    "Implementação de algoritmos de parsing para compiladores em Pascal",
    "Otimização de redes neurais para sistemas embarcados de baixa potência",
    "Análise de sistemas de informação com metodologia estruturada",
    "Fine-tuning de LLMs para tarefas de tradução em português",
    "Knowledge graphs para recomendação de produtos em e-commerce",
    "Análise de sentimentos em redes sociais utilizando BERT",
    "Desenvolvimento de chatbots inteligentes com GPT-3",
    "Avaliação de técnicas de segurança em redes de computadores",
    "Estudo sobre algoritmos de compressão de dados para grandes volumes",
    "Aplicações de aprendizado por reforço em jogos digitais",
    "Técnicas de visualização de dados para análise exploratória",
    "Desenvolvimento de sistemas distribuídos com microserviços",
    "Análise de desempenho de bancos de dados NoSQL em aplicações web",
    "Estudo sobre algoritmos de criptografia para segurança de dados",
    "Implementação de redes neurais convolucionais para reconhecimento de imagens",
    "Desenvolvimento de aplicações móveis com Flutter",
    "Técnicas de mineração de dados para detecção de fraudes financeiras",
    "Estudo sobre a evolução dos sistemas operacionais ao longo das décadas",
    "Fine-tuning de LLMs para tarefas de tradução em português",
    "knowledge distillation em modelos de linguagem grande",
    "Compact models for edge AI applications",
    "Analysis of Criminal Patterns in Police Report Narratives using Spectral Clustering with K-means",
    "Knowledge Distillation in Compact Models: An Approach Applied to Text Processing for Public Security",
    "Inteligência Artificial e sociedade: avanços e riscos"
]

novos_embeddings = embedding_model.encode(novos_titulos)

stylistic_features = np.array([calcular_features_estilo(novos_titulos) for novos_titulos in tqdm(novos_titulos)])
scaler = StandardScaler()
stylistic_features_scaled = scaler.fit_transform(stylistic_features)
novos_embeddings = np.concatenate([novos_embeddings, stylistic_features_scaled], axis=1)


predicoes_prob = classificador.predict(novos_embeddings)

# Resultados
for i, titulo in enumerate(novos_titulos):
    prob = predicoes_prob[i][0]
    
    if prob > 0.5:
        # Classe 1
        classificacao = "ANTES de 2023"
        confianca = prob * 100
    else:
        # Classe 0
        classificacao = "2023 ou DEPOIS"
        confianca = (1 - prob) * 100
        
    print(f"Título: '{titulo}'")
    print(f"  -> Classificação: {classificacao} (Confiança: {confianca:.2f}%)\n")

100%|██████████| 25/25 [00:00<00:00, 292.33it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
Título: 'Um estudo sobre o impacto da IA generativa no desenvolvimento de software'
  -> Classificação: 2023 ou DEPOIS (Confiança: 91.57%)

Título: 'Implementação de algoritmos de parsing para compiladores em Pascal'
  -> Classificação: ANTES de 2023 (Confiança: 99.92%)

Título: 'Otimização de redes neurais para sistemas embarcados de baixa potência'
  -> Classificação: ANTES de 2023 (Confiança: 76.72%)

Título: 'Análise de sistemas de informação com metodologia estruturada'
  -> Classificação: ANTES de 2023 (Confiança: 100.00%)

Título: 'Fine-tuning de LLMs para tarefas de tradução em português'
  -> Classificação: 2023 ou DEPOIS (Confiança: 93.75%)

Título: 'Knowledge graphs para recomendação de produtos em e-commerce'
  -> Classificação: ANTES de 2023 (Confiança: 97.79%)

Título: 'Análise de sentimentos em redes sociais utilizando BERT'
  -> Classificação: ANTES de 2023 (Confiança: 99.87%)

Título: 'Desenvolvimento de chatbots inteligentes com 